In [141]:
import sys

In [160]:
def read_input(file_name):
    with open(file_name, 'r') as file:
        inputs = file.read().splitlines()
        inputs.reverse()
        return [list(i) for i in inputs]

In [234]:
def get_loop(file_name):
    pipe_map = read_input(file_name)

    index_of_s = [(row_string.index('S'), i) for i, row_string in enumerate(pipe_map) if 'S' in row_string][0]
    
    for direction in [[0, 1], [0, -1], [1, 0], [-1, 0]]:
        loop = {index_of_s: 'S'}
        valid_dir, loop = check_dir(pipe_map, index_of_s, direction, loop)
        if valid_dir:
            return loop

pipe_values = {
    '|': [[ 0, 1], [0, -1]],
    '-': [[ 1, 0], [-1, 0]],
    'L': [[-1, 0], [0, -1]],
    'J': [[ 1, 0], [0, -1]],
    '7': [[ 1, 0], [0,  1]],
    'F': [[-1, 0], [0,  1]],
    '.': []
    }

def get_pipe(pipe_map, index):
    return pipe_map[index[1]][index[0]]

def valid_index(pipe_map, index):
    return 0 <= index[0] < len(pipe_map[0]) and 0 <= index[1] < len(pipe_map)

def check_dir(pipe_map, cur_index, direction, loop = {}):
    if len(loop) % 1000 == 0:
        print(len(loop))
    next_index = tuple(i + d for i, d in zip(cur_index, direction))

    if next_index in loop:
        return True, loop

    if not valid_index(pipe_map, next_index):
        return False, loop
    
    pipe = get_pipe(pipe_map, next_index)

    if pipe not in pipe_values or direction not in pipe_values[pipe]:
        return False, loop
    
    loop[next_index] = pipe

    next_direction = pipe_values[pipe][(pipe_values[pipe].index(direction) + 1) % 2]
    next_direction = [-x for x in next_direction]

    return check_dir(pipe_map, next_index, next_direction, loop)

In [142]:
sys.setrecursionlimit(100000)
loop = get_loop('input.txt')

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


In [145]:
loop = get_loop('sample_input2.txt')
loop

{(1, 7): 'S',
 (1, 6): '|',
 (1, 5): '|',
 (1, 4): '|',
 (1, 3): '|',
 (1, 2): '|',
 (1, 1): 'L',
 (2, 1): '-',
 (3, 1): '-',
 (4, 1): 'J',
 (4, 2): '|',
 (4, 3): '7',
 (3, 3): '-',
 (2, 3): 'L',
 (2, 4): '|',
 (2, 5): '|',
 (2, 6): 'F',
 (3, 6): '-',
 (4, 6): '-',
 (5, 6): '-',
 (6, 6): '-',
 (7, 6): '7',
 (7, 5): '|',
 (7, 4): '|',
 (7, 3): 'J',
 (6, 3): '-',
 (5, 3): 'F',
 (5, 2): '|',
 (5, 1): 'L',
 (6, 1): '-',
 (7, 1): '-',
 (8, 1): 'J',
 (8, 2): '|',
 (8, 3): '|',
 (8, 4): '|',
 (8, 5): '|',
 (8, 6): '|',
 (8, 7): '7',
 (7, 7): '-',
 (6, 7): '-',
 (5, 7): '-',
 (4, 7): '-',
 (3, 7): '-',
 (2, 7): '-'}

In [272]:
def set_pipe(pipe_map, index, value):
    pipe_map[index[1]][index[0]] = value

def define_direct_inside(pipe_map, loop):
    cur_index = find_starting_index(pipe_map, loop)

    dir_facing = [[1,1]]
    dir_moving = [0, 1]

    i = 0
    while i < len(loop):
        cur_index = tuple(i + d for i, d in zip(cur_index, dir_moving))

        pipe = get_pipe(pipe_map, cur_index)
        dir_facing, dir_moving = get_direction_facing_and_moving(pipe, dir_facing, dir_moving)
        dir_moving = [-x for x in dir_moving]

        facing_index = [tuple(i + d for i, d in zip(cur_index, dir_facing_i)) for dir_facing_i in dir_facing]

        for f_index in facing_index:
            if f_index not in loop and valid_index(pipe_map, f_index):
                set_pipe(pipe_map, f_index, 'I')       

        i += 1
    
    return pipe_map


def find_starting_index(pipe_map, loop):
    for col in range(len(pipe_map[0])):
        for row in range(len(pipe_map)):
            if (row, col) in loop:
                return (row, col)

pipe_values = {
    '|': [[ 0, 1], [0, -1]],
    '-': [[ 1, 0], [-1, 0]],
    'L': [[-1, 0], [0, -1]],
    'J': [[ 1, 0], [0, -1]],
    '7': [[ 1, 0], [0,  1]],
    'F': [[-1, 0], [0,  1]],
    '.': []
    }

def get_direction_facing_and_moving(pipe, dir_facing, dir_moving):
    if pipe == 'S':
        pipe = '|'
    dir_facing = [[f - m for f, m in zip(fs, dir_moving)] for fs in dir_facing]
    dir_moving = pipe_values[pipe][(pipe_values[pipe].index(dir_moving) + 1) % 2]
    if pipe == '|':
        return [[dir_facing[0][0], 0]], dir_moving
    elif pipe == '-':
        return [[0, dir_facing[0][1]]], dir_moving
    
    if pipe == 'L':
        problem_indices = [[-1, -1], [1,-1], [0, -1], [-1, 0], [-1, 1]]
        if any([x in dir_facing for x in problem_indices]):
            return problem_indices, dir_moving
        else:
            return [[1,1]], dir_moving
    elif pipe == 'J':
        problem_indices = [[1,-1], [1, 0], [1, 1], [0, -1], [-1, -1]]   
        if any([x in dir_facing for x in problem_indices]):
            return problem_indices, dir_moving
        else:
            return [[-1,1]], dir_moving
    elif pipe == '7':
        problem_indices = [[1,1], [0, 1], [-1, 1], [1, 0], [1, -1]]   
        if any([x in dir_facing for x in problem_indices]):
            return problem_indices, dir_moving
        else:
            return [[-1,-1]], dir_moving
    elif pipe == 'F':
        problem_indices = [[-1, 1], [-1,-1], [-1, 0],  [0, 1], [1, 1]]   
        if any([x in dir_facing for x in problem_indices]):
            return problem_indices, dir_moving
        else:
            return [[1,-1]], dir_moving
    

In [258]:
def define_inside(file_name):
    pipe_map = read_input(file_name)
    loop = get_loop(file_name)

    pipe_map = define_direct_inside(pipe_map, loop)
    for col in range(len(pipe_map)):
        for row in range(len(pipe_map[0])):
            if get_pipe(pipe_map, (row, col)) == 'I':
                change_surrounding_to_I(pipe_map, loop, (row + 1, col))
    
    return pipe_map

def change_surrounding_to_I(pipe_map, loop, index):
    if get_pipe(pipe_map, index) == 'I' or index in loop:
        return
    

    set_pipe(pipe_map, index, 'I')

    for d in [[1,0], [-1, 0], [0, 1], [0, -1]]:
        next_index = tuple(i + d for i, d in zip(index, d))
        change_surrounding_to_I(pipe_map, loop, next_index)




In [273]:
results = define_inside('input.txt')
sum([result.count('I') for result in results])

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
L (105, 1)
| [[1, 1]] [0, 1]
7 [[1, 0]] [0, 1]
L [[1, 1], [0, 1], [-1, 1], [1, 0], [1, -1]] [-1, 0]
F [[1, 1]] [0, 1]
J [[1, -1]] [1, 0]
| [[1, -1], [1, 0], [1, 1], [0, -1], [-1, -1]] [0, 1]
7 [[1, 0]] [0, 1]
L [[1, 1], [0, 1], [-1, 1], [1, 0], [1, -1]] [-1, 0]
F [[1, 1]] [0, 1]
J [[1, -1]] [1, 0]
7 [[1, -1], [1, 0], [1, 1], [0, -1], [-1, -1]] [0, 1]
L [[1, 1], [0, 1], [-1, 1], [1, 0], [1, -1]] [-1, 0]
F [[1, 1]] [0, 1]
J [[1, -1]] [1, 0]
7 [[1, -1], [1, 0], [1, 1], [0, -1], [-1, -1]] [0, 1]
- [[1, 1], [0, 1], [-1, 1], [1, 0], [1, -1]] [-1, 0]
L [[0, 1]] [-1, 0]
F [[1, 1]] [0, 1]
- [[1, -1]] [1, 0]
- [[0, -1]] [1, 0]
J [[0, -1]] [1, 0]
7 [[1, -1], [1, 0], [1, 1], [0, -1], [-1, -1]] [0, 1]
- [[1, 1], [0, 1], [-1, 1], [1, 0], [1, -1]] [-1, 0]
- [[0, 1]] [-1, 0]
L [[0, 1]] [-1, 0]
F [[1, 1]] [0, 1]
- [[1, -1]] [1, 0]
- [[0, -1]] [1, 0]
J [[0, -1]] [1, 0]
7 [[1, -1], [1, 0], [1, 1], [0, -1], [-1, -1]] [0, 1]
- [[1, 1], [0

325

In [242]:
test = read_input('sample_input3.txt')
[''.join(t) for t in test]

['L7JLJL-JLJLJL--JLJ.L',
 'L.L7LFJ|||||FJL7||LJ',
 '7-L-JL7||F7|L7F-7F7|',
 '|FFJF7L7F-JF7|JL---7',
 '|F|F-JF---7F7-L7L|7|',
 'L---JF-JLJ.||-FJLJJ7',
 'F--JF--7||LJLJ7F7FJ-',
 'FL-7LJLJ||||||LJL-77',
 'L|LJ||||||||||||F--J',
 'FF7FSF7F7F7F7F7F---7']